# Meter Simulator
This notebook keeps generating values for a power meter, and sends them through a RabbitMQ channel with a frequency.

In [ ]:
%reload_ext autoreload
%autoreload 2
from meter import Meter ## Meter class
import pika, time
from datetime import datetime as dt, timedelta
import shared_params as params

## Setup Parameters
Set the meter parameters

In [ ]:
random_seed = None        # random seed for reproducible results
meter_range = (0,9000)    # the range for the power meter.

## Start Meter

Initialize Meter

In [ ]:
meter = Meter( 
    random_seed=random_seed,
    meter_range=meter_range
)
start_time = dt.now()
## send file name to be saved to the 
file_name=str(start_time.year)+"-"+str(start_time.month)+"-"\
        +str(start_time.day)+"-"+str(start_time.hour) +"-"+str(start_time.minute)+".csv"
meter.send_init(
    file_name=file_name
)
print("File name to be saved:",file_name)

Determine the time frame to run, and start the reading and sending the values of the meter.

In [ ]:
now = start_time = dt.now()
until = now +timedelta(minutes=1,days=0)

while now < until:
    meter.process_meter(now.strftime(params.datetime_format),0)
    now = dt.now()
    time.sleep(1.0 - (now - start_time).seconds % 1.0)
meter.process_meter(now.strftime(params.datetime_format),1)